# Baseline model
### "Capybaras" capstone image recognition project


Install PIL - tool for image processing.
#!pip install pillow

Import all necessary tools.



In [1]:
from PIL import Image, ImageOps
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

1. Load the data

In [8]:
# Paths
dataset_folder = '../data/new_train_features'  # Path to the folder with images
labels_file = '../data/train_labels.csv'  # Path to the CSV file with labels

# Path to the train folder and the new train folder
input_train_folder = '../data/train_features'
output_train_folder = '../data/new_train_features'

# Path to the test folder and the new test folder
input_test_folder = '../data/test_features'
output_test_folder = '../data/new_test_features'

# Load labels from CSV
df = pd.read_csv(labels_file)

frac = 1.0

df = df.sample(frac=frac, random_state=1) # Load a random fraction of the data

FileNotFoundError: [Errno 2] No such file or directory: '../data/train_labels.csv'

2. Preprocess the data. 
Below we defined the functions to preprocess data for our model.


In [ ]:
def max_sizes(input_folder):
    """Calculates the maximum width and height of all images in the input folder.

    Args:
        input_folder (str): Path to the folder containing the images.

    Returns:
        tuple: Maximum width and height of the images found in the folder.
    """
    # Initialize maximum width and height
    max_width = 0
    max_height = 0
    
    # Iterate through all files in the input folder
    for filename in os.listdir(input_folder):
        # Check if the file is an image with a supported extension
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Construct the full file path
            file_path = os.path.join(input_folder, filename)
            # Open the image file
            with Image.open(file_path) as img:
                # Get the width and height of the image
                width, height = img.size
                # Update max_width if the current image's width is greater
                if width > max_width:
                    max_width = width
                # Update max_height if the current image's height is greater
                if height > max_height:
                    max_height = height
    # Return the maximum width and height found
    return max_width, max_height

# Define the range of white color
def is_white_color(color, threshold=230):
    # Check if color is close to white within the given threshold
    return all(c >= threshold for c in color)


# Adjust threshold based on what is considered "close to white"
def crop_white_bottom_from_image(img, height_to_check=8, check_width=10, start_from=60, threshold=230):
    width, height = img.size
    # Define the region to check for white color
    box = (start_from, height - height_to_check, start_from + check_width, height)
    region = img.crop(box)
    # Convert the region to a NumPy array for easy processing
    region_np = np.array(region)
    # Check if all pixels in the region are close to white
    white_pixels = np.apply_along_axis(is_white_color, 1, region_np)
    
    if np.all(white_pixels):
        # Crop the bottom 16 pixels
        img = img.crop((0, 0, width, height - height_to_check - 8))
    return img


def padding_images(img,max_width,max_height):
    # Calculate padding required to center the image
    delta_width = max_width - img.width
    delta_height = max_height - img.height
    padding = (delta_width // 2, delta_height // 2, delta_width - (delta_width // 2), delta_height - (delta_height // 2))
    # Add padding and create a new image with the desired size
    return ImageOps.expand(img, padding, fill=0)

def format_to_rgb(img):
    if img.mode != 'RGB':
        # Convert image to RGB
        img = img.convert('RGB')
    return img

def crop_white_bottom_add_padding_all_rgb(input_folder,output_folder):
    max_width, max_height = max_sizes(input_folder)

    # Create the new folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    # Iterate through each image in the source folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(input_folder, filename)
            with Image.open(file_path) as img:
                img = crop_white_bottom_from_image(img)
                img = padding_images(img,max_width,max_height)
                img = format_to_rgb(img)
                # Save the original image to the new folder
                new_file_path = os.path.join(output_folder, filename)
                img.save(new_file_path)
    print(output_folder, "Processing complete.")
    

In [ ]:
#actual preprocessing
crop_white_bottom_add_padding_all_rgb(input_train_folder,output_train_folder)
crop_white_bottom_add_padding_all_rgb(input_test_folder,output_test_folder)

# Image parameters
img_width, img_height = max_sizes(dataset_folder)
#img_height = 128  # Set your image height
#img_width = 128   # Set your image width
batch_size = 32

# Add a full path to each image in the dataframe
df['id'] = df['id'].apply(lambda x: os.path.join(dataset_folder, x+'.jpg'))

# Data generator
datagen = ImageDataGenerator(rescale=1./255)  # Step 1: Normalize pixel values to the range [0, 1]

# Create the training generator
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,  # Step 2: Use training DataFrame
    directory=None,  # Images paths are specified in the DataFrame
    x_col='id',  # Column with image file paths
    y_col=train_df.columns[1:],  # All columns except 'ID' are considered as labels
    target_size=(img_height, img_width),  # Step 3: Resize images to specified dimensions
    batch_size=batch_size,  # Step 4: Number of images to process in each batch
    class_mode='raw'  # Step 5: Output labels as they are, without any transformation
)

# Create the validation generator
val_generator = datagen.flow_from_dataframe(
    dataframe=val_df,  # Step 6: Use validation DataFrame
    directory=None,  # Images paths are specified in the DataFrame
    x_col='id',  # Column with image file paths
    y_col=val_df.columns[1:],  # All columns except 'ID' are considered as labels
    target_size=(img_height, img_width),  # Step 7: Resize images to specified dimensions
    batch_size=batch_size,  # Step 8: Number of images to process in each batch
    class_mode='raw'  # Step 9: Output labels as they are, without any transformation
)

"""# Example: Display a batch of images and labels
images, labels = next(train_generator)  # Step 10: Get the next batch of images and labels from the training generator
plt.figure(figsize=(12, 12))  # Step 11: Create a new figure for plotting
for i in range(9):  # Step 12: Loop over the first 9 images in the batch
    plt.subplot(3, 3, i + 1)  # Step 13: Define a 3x3 grid of subplots
    plt.imshow(images[i])  # Step 14: Display the i-th image
    plt.title(f"Labels: {labels[i]}")  # Step 15: Add title with corresponding labels
    plt.axis('off')  # Step 16: Hide the axis
plt.show()  # Step 17: Show the plot with images"""

3. Train/Test-split of the data

In [ ]:
# Perform stratified split based on the labels
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df[df.columns[1:]])


4. Build model

In [ ]:
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(16, activation='relu'),
    Dense(len(df.columns) - 1, activation='softmax')  # Output layer for multi-label classification
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

5. Train model

In [ ]:
history = model.fit(train_generator, validation_data=val_generator, epochs=10)

In [2]:

# Load the saved model
with tf.device('/cpu:0'):
    new_model = tf.keras.models.load_model('/Users/alexandersimakov/Documents/CAPSTONE/ds-capstone-project/models/firstCNNmodel')

# Check its architecture
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 538, 958, 16)      448       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 269, 479, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 267, 477, 16)      2320      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 133, 238, 16)      0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 506464)            0         
                                                                 
 dense_4 (Dense)             (None, 16)               

6. Evaluation

6.1 Accuracy


In [6]:

# Print the accuracy for each epoch
print("Training accuracy for each epoch:", history.history['accuracy'])
print("Validation accuracy for each epoch:", history.history['val_accuracy'])

NameError: name 'train_generator' is not defined

6.2 Loss vs Epoch

In [ ]:
def evaluation_metric(model, saving = False):

    """
    Plots and (optionally) saves accuracy and loss of the model per epoch.
    Args:
        model: The trained model object
        saving = False: if True, it asks for user input of a filename. The accuracies and losses are saved as a csv file in the current directory.
    Returns:
        plots of Training and Validation Accuracy
        saves result as csv file (optional)
    """

    # create variable for different accuracy metrics vs. epochs
    accuracy = model.history['accuracy']
    val_accuracy = model.history['val_accuracy']
    loss = model.history['loss']
    val_loss = model.history['val_loss']

    # plot Accuracy
    fig1, ax1 = plt.subplots()
    ax1.plot(accuracy,label='train_accuracy')
    ax1.plot(val_accuracy,label='val_accuracy')
    ax1.set_xlabel('epochs')
    ax1.set_ylabel('accuracy')
    ax1.legend()
    fig1.show()

    # plot Loss
    fig2, ax2 = plt.subplots()
    ax2.plot(loss,label='train_loss')
    ax2.plot(val_loss,label='val_loss')
    ax2.set_xlabel('epochs')
    ax2.set_ylabel('loss')
    ax2.legend()
    fig2.show()

    # saving the metrics in a csv-file
    if saving == True:
        d = {'accuracy': accuracy, 'val_accuracy': val_accuracy, 'loss': loss, 'val_loss': val_loss}
        df = pd.DataFrame(d)
        name = input("Please enter the filename (without .csv): ")
        name = name + '.csv'
        df.to_csv(name)

evaluation_metric(history, saving = False)

6.3 Confusion Matrix

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Step 1: Get Predictions on Validation Data
val_preds = model.predict(val_generator)


# Convert predictions to class labels
val_preds_classes = np.argmax(val_preds, axis=1)

# Step 2: Get True Labels
# Extract true labels from the validation generator
val_true_labels = val_generator.labels

# Manually specify class labels
class_labels = ['antelope_duiker', 'bird', 'blank', 'civet_genet', 'hog', 'leopard', 'monkey_prosimian', 'rodent']  # Adjust as necessary

# Compute the confusion matrix
cm = confusion_matrix(val_true_labels, val_preds_classes)

# Display the confusion matrix
fig, ax = plt.subplots(figsize=(10, 10))
ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels).plot(ax=ax, xticks_rotation=90, colorbar=True)
plt.show()